# Convertidor de Audio a Texto

Importar todas las líbrerias a utilizar

In [ ]:
#Importar librerías
import speech_recognition as sr
import os
from pydub import AudioSegment
from tqdm import tqdm
from pydub.silence import split_on_silence

Convertir audios de una carpeta desde mp3 a wav

In [ ]:
#Directorio con audios
path_mp3 = '../data/raw/'
path_wav = '../data/interim/''

#Cambiar directorio de trabajo
os.chdir(path_mp3)

#Listado de archivos
directorio = os.listdir()

#Convertir mp3 a wav
for file in tqdm(directorio):
    name, ext = os.path.splitext(file)
    #Solo para los archivos con extensión mp3
    if ext == ".mp3":    
        audio_mp3 = AudioSegment.from_file(file)
        audio_mp3.export(path_wav+"{0}.wav".format(name), format="wav") 


Convertir todos los audios de una carpeta a textos, utilizando chunks

In [ ]:
#Iniciar reconocedor de voz
reconocedor = sr.Recognizer()

#Directorio con audios
path_wav = './data/interim/'
path_textos = './data/processed/'

#Cambiar directorio de trabajo
os.chdir(path_wav)

#Listado de archivos
directorio = os.listdir()

##Leer archivos wav
for file in tqdm(directorio):
    name, ext = os.path.splitext(file)
    #Solo para los archivos con extensión wav
    if ext == ".wav":    
        #Definir archivo de audio
        audio_file = AudioSegment.from_file(file)
        
        # hacer split al audio cuando el silencio sea de más de 700 milisegundos
        chunks = split_on_silence(audio_file, min_silence_len = 500, silence_thresh = audio_file.dBFS-14, keep_silence=500)
        
        carpeta_chunks = "audio-chunks"
        
        # crear directorio para guardar los chunks de audios
        if not os.path.isdir(carpeta_chunks):
            os.mkdir(carpeta_chunks)
        
        texto_completo = ""
        
        # procesar cada chunk 
        for i, audio_chunk in enumerate(chunks, start=1):
            # exportar audio chunk y guardarlo en carpeta
            archivo_con_chunks = os.path.join(carpeta_chunks, f"chunk{i}.wav")
            audio_chunk.export(archivo_con_chunks, format="wav")
            #Reconocedor de voz
            with sr.AudioFile(archivo_con_chunks) as source:
                reconocedor.adjust_for_ambient_noise(source) #Se ajusta para eliminar sonido ambiente
                audio = reconocedor.record(source) #Se lee el audio
                try:
                    texto = reconocedor.recognize_google(audio, language='es-CL') #Texto reconocido
                    #Exportar resultado a archivo de texto   
                except sr.UnknownValueError:
                       print("No entendí el audio")
                except sr.RequestError as e:
                       print("No obtuve ningun requerimiento {0}".format(e))
                else:
                    texto = f"{texto.capitalize()}. "
                    texto_completo += texto
                    with open(path_textos+"{0}.txt".format(name),mode = 'w') as file:
                        file.write(texto_completo)
                        file.write("/n")
        
